In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install transformers

In [ ]:
pip install SentencePiece

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Define your KaNet model
class KaNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(KaNet, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        output = self.fc(x)
        return output

# Load your data from a .npy file
data = np.load("/content/drive/MyDrive/BrainTeaser/word_puzzle.npy", allow_pickle=True)[()]
questions = [entry['question'] for entry in data]
options = [entry['choice_list'] for entry in data]
correct_indices = [entry['label'] for entry in data]

# Preprocess and vectorize your data
vectorizer = TfidfVectorizer()  # You may need to customize this for your data
X = vectorizer.fit_transform(questions)
input_dim = X.shape[1]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, correct_indices, test_size=0.2)

# Create labels tensor
train_labels = torch.tensor(y_train, dtype=torch.long)  # Convert labels to Long
val_labels = torch.tensor(y_val, dtype=torch.long)  # Convert labels to Long

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create TensorDatasets
train_dataset = TensorDataset(torch.tensor(X_train.toarray(), dtype=torch.float), train_labels)
val_dataset = TensorDataset(torch.tensor(X_val.toarray(), dtype=torch.float), val_labels)

# Define a data loader
batch_size = 16  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define your KaNet model
hidden_dim = 128  
output_dim = 4  

model = KaNet(input_dim, hidden_dim, output_dim)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

num_epochs = 1
model.to(device)

# Training loop
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(train_loader, 0):
        inputs, labels = batch[0].to(device), batch[1].to(device)  # Move input data to the same device

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / (i + 1)}")

print("Finished Training")


In [ ]:
model.eval()  # Switch to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for batch in val_loader:
        inputs, labels = batch[0].to(device), batch[1].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Validation Accuracy: {accuracy:.2f}%')
